In [ ]:
#CNN 학습
import numpy as np
import pickle #텍스트 이외의 자료형을 파일로 저장하기 위해 제공하는 파이썬 모듈
import cv2, os
from glob import glob # 파일들의 목록을 뽑을때 사용
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras import backend as K
K.set_image_data_format('channels_last')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def get_image_size():
   img = cv2.imread('gestures/1/100.jpg', 0)
   return img.shape

def get_num_of_classes(): #class 수에따라 dense 지정해주기 위해서
   return len(glob('gestures/*'))

image_x, image_y = get_image_size()

def cnn_model():
   num_of_classes = get_num_of_classes()
   model = Sequential()
   model.add(Conv2D(16, (2,2), input_shape=(image_x, image_y, 1), activation='relu'))
   model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
   model.add(Conv2D(32, (3,3), activation='relu'))
   model.add(MaxPooling2D(pool_size=(3, 3), strides=(3, 3), padding='same'))
   model.add(Conv2D(64, (5,5), activation='relu'))
   model.add(MaxPooling2D(pool_size=(5, 5), strides=(5, 5), padding='same'))
   model.add(Flatten())
   model.add(Dense(128, activation='relu'))
   model.add(Dropout(0.2))
   model.add(Dense(get_num_of_classes()+1, activation='softmax'))
   model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   filepath="cnn_model.h5"
   checkpoint1 = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
   callbacks_list = [checkpoint1]
   return model, callbacks_list

def train():
   with open("train_images", "rb") as f:
      train_images = np.array(pickle.load(f))
   with open("train_labels", "rb") as f:
      train_labels = np.array(pickle.load(f), dtype=np.int32)

   with open("val_images", "rb") as f:
      val_images = np.array(pickle.load(f))
   with open("val_labels", "rb") as f:
      val_labels = np.array(pickle.load(f), dtype=np.int32)
        
    #데이터 전처리
   train_images = np.reshape(train_images, (train_images.shape[0], image_x, image_y, 1))
   val_images = np.reshape(val_images, (val_images.shape[0], image_x, image_y, 1))
    # one-hot-encode
   #train_labels = np_utils.to_categorical(train_labels)
   #val_labels = np_utils.to_categorical(val_labels)

   print(val_labels.shape) # 테스트값 결과 출력

   model, callbacks_list = cnn_model()
   model.summary()
 
   model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=30, batch_size=12, callbacks=callbacks_list)
   scores = model.evaluate(val_images, val_labels, verbose=0)
   print("CNN Error: %.2f%%" % (100-scores[1]*100))
   model.save('cnn_model.h5')

train()
K.clear_session();